# 19 卷积层
### 两个原则
- 平移不变性：在图片的任意像素都可以识别，卷积层的输出结果不会变化
- 局部性：只需要看到一个小块就可以识别
### 重新考察全连接层
- 将输入和输出变形为一个矩阵
- 将权重变形为4维张量
- `V`是`W`的重新索引
![](./assets/2022-02-09-16-53-00.png)
卷积就是weight share 就是全连接
![](./assets/2022-02-09-16-54-53.png)
也就是说，卷积核不能太大。
对全连接层使用平移不变性和局部性得到卷积层
![](./assets/2022-02-09-16-57-30.png)
### 二维交叉相关（二维卷积）
![](./assets/2022-02-09-17-06-02.png)
### 二维卷积层
![](./assets/2022-02-09-17-07-43.png)
![](./assets/2022-02-09-17-11-26.png)
### 总结
- 卷积层将输入和核矩阵进行交叉相关，加上偏移后得到输出
- 核矩阵和偏移是可学习的参数
- 核矩阵的大小是超参数


In [1]:
#互相关运算
import torch
from torch import nn
from d2l import torch as d2l

def corr2d(X, K):  
    """计算二维互相关运算。"""
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return Y

In [2]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

In [4]:
#实现二维卷积层
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

In [5]:
#卷积层的一个简单应用： 检测图像中不同颜色的边缘
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [6]:
K = torch.tensor([[1.0, -1.0]])

In [7]:
#输出Y中的1代表从白色到黑色的边缘，-1代表从黑色到白色的边缘
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [8]:
#卷积核K只可以检测垂直边缘
corr2d(X.t(), K)


tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [11]:
#学习由X生成Y的卷积核
conv2d = nn.Conv2d(1, 1, kernel_size=(1, 2), bias=False)

X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y)**2
    conv2d.zero_grad()
    l.sum().backward()
    conv2d.weight.data[:] -= 3e-2 * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'batch {i+1}, loss {l.sum():.3f}')

batch 2, loss 13.280
batch 4, loss 4.396
batch 6, loss 1.626
batch 8, loss 0.637
batch 10, loss 0.256


In [12]:
#所学的卷积核的权重张量
conv2d.weight.data.reshape((1, 2))

tensor([[ 1.0411, -0.9373]])